# EDA for Prototype's Fire Data
#### Hannah Catri 217569841




In [1]:
import pandas as pd
from sodapy import Socrata
import datetime
from pandas_profiling import ProfileReport
import requests
import json

## Emergency Services Data Stream

#### Fire Department Calls for Service

In [2]:
# Get the number of rows in the dataset
# Save as a variable in case paging method of data retireval is needed
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)

The number of incidents in the set is 5649191


In [3]:
%%time
# Configure the Socrata client and create the generator that will retrieve the data
client = Socrata("data.sfgov.org", None)
dataset_gen=client.get("nuek-vuh3", limit=2000)
# Extract the dataset, this may take a while.
# Use shape to confirm the size matches the rows as per above maxNumber.
fire_cfs_df = pd.DataFrame.from_records(dataset_gen)
fire_cfs_df.shape
fire_cfs_df.sample(5)

CPU times: user 77.9 ms, sys: 8.83 ms, total: 86.8 ms
Wall time: 1.29 s


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,call_type_group,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,:@computed_region_ajp5_b2md
414,210552675,RC4,21024660,Traffic Collision,2021-02-24T00:00:00.000,2021-02-24T00:00:00.000,2021-02-24T18:02:09.000,2021-02-24T18:06:49.000,2021-02-24T18:07:50.000,2021-02-24T18:10:59.000,...,Potentially Life-Threatening,1,RESCUE CAPTAIN,3,9,11,Outer Mission,210552675-RC4,"{'type': 'Point', 'coordinates': [-122.4429273...",28
124,210370693,B06,21016668,Alarms,2021-02-06T00:00:00.000,2021-02-06T00:00:00.000,2021-02-06T08:34:46.000,2021-02-06T08:36:06.000,2021-02-06T08:36:14.000,2021-02-06T08:38:00.000,...,Alarm,1,CHIEF,2,6.0,9,Bernal Heights,210370693-B06,"{'type': 'Point', 'coordinates': [-122.4182024...",2
1800,211581595,D2,21067466,Structure Fire,2021-06-07T00:00:00.000,2021-06-07T00:00:00.000,2021-06-07T12:40:50.000,2021-06-07T12:41:51.000,2021-06-07T12:42:16.000,2021-06-07T12:42:52.000,...,Alarm,1,CHIEF,8,7,5,Inner Sunset,211581595-D2,"{'type': 'Point', 'coordinates': [-122.4729252...",14
288,210553275,QRV1,21024727,Medical Incident,2021-02-24T00:00:00.000,2021-02-24T00:00:00.000,2021-02-24T21:54:01.000,2021-02-24T21:56:26.000,2021-02-24T21:56:49.000,2021-02-24T21:56:54.000,...,Potentially Life-Threatening,1,SUPPORT,1,3,6,South of Market,210553275-QRV1,"{'type': 'Point', 'coordinates': [-122.4090260...",34
1064,211081981,E08,21046762,Medical Incident,2021-04-18T00:00:00.000,2021-04-18T00:00:00.000,2021-04-18T16:02:36.000,2021-04-18T16:02:36.000,2021-04-18T16:03:00.000,2021-04-18T16:04:16.000,...,Potentially Life-Threatening,1,ENGINE,1,2,6,South of Market,211081981-E08,"{'type': 'Point', 'coordinates': [-122.4110969...",34


In [4]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=7)).isoformat()
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"+str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
#print(query_str)
data=pd.read_json(query_str)
print(data.shape)
data.sample(5)

(1000, 34)


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,transport_dttm,hospital_dttm
708,212460317,E09,21105715,Medical Incident,2021-09-03T00:00:00.000,2021-09-02T00:00:00.000,2021-09-03T04:07:43.000,2021-09-03T04:09:06.000,2021-09-03T04:09:15.000,2021-09-03T04:11:57.000,...,1,ENGINE,1,10,10,Bayview Hunters Point,212460317-E09,"{'type': 'Point', 'coordinates': [-122.4035072...",NaN,NaN
213,212463033,52,21106300,Medical Incident,2021-09-03T00:00:00.000,2021-09-03T00:00:00.000,2021-09-03T18:52:45.000,2021-09-03T18:52:45.000,2021-09-03T18:53:50.000,2021-09-03T18:53:53.000,...,1,MEDIC,2,9,11,Excelsior,212463033-52,"{'type': 'Point', 'coordinates': [-122.4409453...",2021-09-03T19:21:00.000,2021-09-03T19:35:14.000
743,212460194,KM04,21105688,Medical Incident,2021-09-03T00:00:00.000,2021-09-02T00:00:00.000,2021-09-03T02:15:35.000,2021-09-03T02:15:35.000,2021-09-03T02:17:21.000,2021-09-03T02:18:03.000,...,1,PRIVATE,1,8.0,4,Sunset/Parkside,212460194-KM04,"{'type': 'Point', 'coordinates': [-122.4864467...",2021-09-03T03:06:46.000,2021-09-03T03:34:18.000
658,212460494,E33,21105739,Structure Fire,2021-09-03T00:00:00.000,2021-09-02T00:00:00.000,2021-09-03T06:46:25.000,2021-09-03T06:47:15.000,2021-09-03T06:47:23.000,2021-09-03T06:48:21.000,...,1,ENGINE,3,9,7,West of Twin Peaks,212460494-E33,"{'type': 'Point', 'coordinates': [-122.4649381...",NaN,NaN
619,212460887,E07,21106058,Medical Incident,2021-09-03T00:00:00.000,2021-09-03T00:00:00.000,2021-09-03T10:11:31.000,2021-09-03T10:11:31.000,2021-09-03T10:11:31.000,2021-09-03T10:11:31.000,...,1,ENGINE,2,6,9,Mission,212460887-E07,"{'type': 'Point', 'coordinates': [-122.4125496...",NaN,NaN


In [5]:
profile = ProfileReport(data, title="Fire Dept Calls for Service")
profile.to_widgets()

Summarize dataset:   0%|          | 0/47 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Twitter Data Stream

#### Tweets with the word 'fire' in region

In [6]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMdDTQEAAAAAfYqWjo5%2BOgmFP%2FmqGGkq74pEahs%3DNqy7E6BdRittRWu2v3DJAVXEvbDLhAejdCBccR4mQPNO70s9Sl'

In [7]:

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    return headers

In [8]:
def create_url(keyword,start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [9]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [10]:
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=2)).isoformat()
#Inputs for the request
headers = create_headers(bearer_token)
keyword = "caldor fire lang:en"
#place = "San Francisco"
start_time = periodstart[:-2]+'Z'
end_time = periodend[:-2]+'Z'
max_results = 100

In [11]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [12]:
tweets=pd.json_normalize(json_response['data'])
tweets.head()

,lang,reply_settings,created_at,source,conversation_id,id,text,author_id,referenced_tweets,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,geo.place_id
0,en,everyone,2021-09-09T16:05:38.000Z,Twitter for iPhone,1435997837808914435,1435997837808914435,RT @sfchronicle: A foil-wrapped home survived ...,705710778,"[{'type': 'retweeted', 'id': '1435729419683852...",1757,0,0,0,NaN
1,en,everyone,2021-09-09T16:05:37.000Z,Twitter Web App,1435997831395954694,1435997831395954694,RT @accuweather: Millions of acres have been s...,48160300,"[{'type': 'retweeted', 'id': '1435996589433278...",3,0,0,0,NaN
2,en,everyone,2021-09-09T16:05:36.000Z,Twitter for iPhone,1435997828573245441,1435997828573245441,RT @wildland_zko: #CaldorFire - 9/8/2021\nOur ...,20941869,"[{'type': 'retweeted', 'id': '1435605059979546...",18,0,0,0,NaN
3,en,everyone,2021-09-09T16:05:14.000Z,Twitter for iPhone,1435997737992867846,1435997737992867846,RT @sfchronicle: A foil-wrapped home survived ...,2459187866,"[{'type': 'retweeted', 'id': '1435729419683852...",1757,0,0,0,NaN
4,en,everyone,2021-09-09T16:04:58.000Z,Twitter Web App,1435997670481403905,1435997670481403905,RT @sfchronicle: A foil-wrapped home survived ...,332856426,"[{'type': 'retweeted', 'id': '1435729419683852...",1757,0,0,0,NaN


In [13]:
profile = ProfileReport(tweets, title="Fire Related Tweets")
profile.to_widgets()

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Climate Data Stream

#### Weather API

Add weather API: https://www.weather.gov/documentation/services-web-api
